In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [61]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [62]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [63]:
all_data = pd.concat([train_data, test_data])
all_data.reset_index(drop=True, inplace=True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


欠損値のあるカラム
- Age：ランダムフォレストで算出
- Fare：同じPClass・Embarkedの行の平均をfill
- Cabin(欠損値多め)：欠損値補完をしない
- Embarked：一番多い値で埋める

In [64]:
# Fareの補完
fare = all_data.loc[(all_data['Embarked'] == 'S') & (all_data['Pclass'] == 3), 'Fare'].mean()
all_data['Fare']=all_data['Fare'].fillna(fare)

In [65]:
from sklearn.ensemble import RandomForestClassifier

# Ageの補完
features = ['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']
age_data = pd.get_dummies(all_data[features])
age_data['Fare'] = age_data['Fare'].astype(int)
know_age_data = age_data[age_data['Age'].notnull()]
unknown_age_data = age_data[age_data['Age'].isnull()]

y = know_age_data['Age'].astype(int)
X = know_age_data.drop('Age', axis=1)
X_test = unknown_age_data.drop('Age', axis=1)

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
model.fit(X, y)
predictions = model.predict(X_test)

all_data.loc[(all_data.Age.isnull()), 'Age'] = predictions

# 年齢別生存曲線と死亡曲線
# facet = sns.FacetGrid(all_data[0:890], hue='Survived', aspect=2)
# facet.map(sns.kdeplot, 'Age')
# facet.set(xlim=(0, all_data.loc[0:890,'Age'].max()))
# facet.add_legend()
# plt.show()

In [66]:
# Embarkedの補完
all_data['Embarked'].fillna('S')

0       S
1       C
2       S
3       S
4       S
       ..
1304    S
1305    C
1306    S
1307    S
1308    C
Name: Embarked, Length: 1309, dtype: object

In [67]:
# Survivedの推定
features = ['Survived', 'Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
sur_data = pd.get_dummies(all_data[features])
sur_data['Fare'] = sur_data['Fare'].astype(int)
know_sur_data = sur_data[sur_data['Survived'].notnull()]
unknown_sur_data = sur_data[sur_data['Survived'].isnull()]

y = know_sur_data['Survived']

X = know_sur_data.drop('Survived', axis=1)
X_test = unknown_sur_data.drop('Survived', axis=1)

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [68]:
output_data = pd.read_csv("/kaggle/working/submission.csv")
output_data.head()

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
